In [1]:
import bittensor as bt

In [2]:
sub = bt.subtensor('ws://104.171.201.172:9944')

In [132]:
sub = bt.subtensor('local')

In [134]:
sub.get_subnets()

[0, 1, 3]

In [22]:
alice = bt.wallet('alice')
bob = bt.wallet('bob')

In [123]:
# Add stake.
sub = bt.subtensor('local')
alice = bt.Keypair.create_from_uri('//Alice')
bob = bt.Keypair.create_from_uri('//Bob')
charlie = bt.Keypair.create_from_uri('//Charlie')
call = sub.substrate.compose_call(
    call_module="SubtensorModule",
    call_function="add_stake",
    call_params={
        "hotkey": charlie.ss58_address,
        "netuid": 1,
        "amount_staked": 1_000_000_000,
    },
)
extrinsic = sub.substrate.create_signed_extrinsic(
    call=call, keypair=alice
)
response = sub.substrate.submit_extrinsic(
    extrinsic,
    wait_for_finalization=True,
)
response.process_events()
if not response.is_success:
    print(response.error_message)
else:
    print ('success')

success


In [10]:
sub = bt.subtensor('local')
alice = bt.Keypair.create_from_uri('//Alice')
bob = bt.Keypair.create_from_uri('//Bob')
call = sub.substrate.compose_call(
    call_module="SubtensorModule",
    call_function="remove_stake",
    call_params={
        "hotkey": bob.ss58_address,
        "netuid": 1,
        "amount_unstaked": 1_000_000_000,
    },
)
extrinsic = sub.substrate.create_signed_extrinsic(
    call=call, keypair=alice
)
response = sub.substrate.submit_extrinsic(
    extrinsic,
    wait_for_finalization=True,
)
response.process_events()
if not response.is_success:
    print(response.error_message)
else:
    print ('success')

{'type': 'Module', 'name': 'UnstakeRateLimitExceeded', 'docs': ['A transactor exceeded the rate limit for unstaking.']}


In [42]:
# Create networks
import bittensor as bt
alice = bt.Keypair.create_from_uri('//Alice')
bob = bt.Keypair.create_from_uri('//Bob')
call = sub.substrate.compose_call(
    call_module="SubtensorModule",
    call_function="register_network",
    call_params={
        "mechid": 1,
        "hotkey": bob.ss58_address,
    },
)
extrinsic = sub.substrate.create_signed_extrinsic(
    call=call, keypair=alice
)
response = sub.substrate.submit_extrinsic(
    extrinsic,
    wait_for_finalization=True,
)
response.process_events()
if not response.is_success:
    print(response.error_message)
else:
    print ('success')

success


In [122]:
# Register keys
import bittensor as bt
sub = bt.subtensor('local')
alice = bt.Keypair.create_from_uri('//Alice')
charlie = bt.Keypair.create_from_uri('//Charlie')
call = sub.substrate.compose_call(
    call_module="SubtensorModule",
    call_function="burned_register",
    call_params={
        "hotkey": charlie.ss58_address,
        "netuid": 1,
    },
)
extrinsic = sub.substrate.create_signed_extrinsic(
    call=call, keypair=alice
)
response = sub.substrate.submit_extrinsic(
    extrinsic,
    wait_for_finalization=True,
)
response.process_events()
if not response.is_success:
    print(response.error_message)
else:
    print ('success')

success


In [135]:
# Get all subnets
sub = bt.subtensor('local')
subnets = sub.get_subnets()

# Query the state for each subnet
from rich.table import Table
from rich.console import Console
table = Table(title="Subnet Information")
table.add_column("Subnet", justify="right", style="cyan", no_wrap=True)
table.add_column("Mechid", style="magenta")
table.add_column("S_TAO", justify="right", style="green")
table.add_column("Alpha In", justify="right", style="yellow")
table.add_column("Alpha Out", justify="right", style="yellow")
table.add_column("Price", justify="right", style="red")
table.add_column("N", justify="right", style="blue")  # Added new column for subnetwork n

for subnet in subnets:
    mechid = sub.substrate.query(
        module="SubtensorModule",
        storage_function="SubnetMechanism",
        params=[subnet]
    ).value
    s_tao = sub.substrate.query(
        module="SubtensorModule",
        storage_function="SubnetTAO",
        params=[subnet]
    ).value/10**9
    alpha_in = sub.substrate.query(
        module="SubtensorModule",
        storage_function="SubnetAlphaIn",
        params=[subnet]
    ).value/10**9
    alpha_out = sub.substrate.query(
        module="SubtensorModule",
        storage_function="SubnetAlphaOut",
        params=[subnet]
    ).value/10**9
    sn = sub.substrate.query(
        module="SubtensorModule",
        storage_function="SubnetworkN",
        params=[subnet]
    ).value
    price = s_tao / alpha_in if alpha_in != 0 else float('inf')
    
    table.add_row(
        str(subnet),
        str(mechid),
        f"{s_tao:,}",
        f"{alpha_in:,}",
        f"{alpha_out:,}",
        f"{price:.2f}" if price != float('inf') else "∞",
        str(sn)  # Added subnetwork n to the table row
    )

console = Console()
console.print(table)




                             Subnet Information                              
┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━┓
┃ Subnet ┃ Mechid ┃   S_TAO ┃        Alpha In ┃       Alpha Out ┃ Price ┃ N ┃
┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━┩
│      0 │ 0      │     0.0 │             0.0 │             0.0 │     ∞ │ 0 │
│      1 │ 1      │ 5,308.0 │ 4,307.218197687 │ 1,768.722246339 │  1.23 │ 2 │
│      3 │ 0      │     0.0 │             0.0 │             0.0 │     ∞ │ 0 │
└────────┴────────┴─────────┴─────────────────┴─────────────────┴───────┴───┘

In [137]:
alice = bt.Keypair.create_from_uri('//Alice')
# charlie = bt.Keypair.create_from_uri('//Charlie')
bob = bt.Keypair.create_from_uri('//Bob')
for i in range(4):
    s_tao = sub.substrate.query(
        module="SubtensorModule",
        storage_function="Alpha",
        params=(alice.ss58_address, bob.ss58_address, i)
    )
    print (s_tao)

0
0
0
0
